In [1]:
!export PYTORCH_CUDA_ALLOC_CONF="expandable_segments:True"

In [2]:
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: problems found:
        - require?  X jupyter-js-widgets/extension


In [3]:
!export TF_CPP_MIN_LOG_LEVEL=2

In [4]:
import gc, torch
gc.collect()
torch.cuda.empty_cache()

In [5]:
import os
import torch
import numpy as np
import pandas as pd
import gc
from datasets import Dataset
from datasets import concatenate_datasets
from transformers import Wav2Vec2FeatureExtractor, AutoConfig, EarlyStoppingCallback
from transformers import Wav2Vec2ForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import librosa
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
import wandb
import torch
from torch.utils.data import Dataset as TorchDataset
from sklearn.metrics import confusion_matrix, roc_curve, auc
import matplotlib.pyplot as plt
from torchinfo import summary
import json
from sklearn.model_selection import KFold
import seaborn as sns
from io import BytesIO
import colorama
from colorama import Fore, Back, Style

# Initialize colorama for cross-platform colored output
colorama.init()

class AudioAugmenter:
    """Audio augmentation techniques with length preservation"""
    @staticmethod
    def pad_or_truncate(audio, max_length=32000):
        if len(audio) > max_length:
            return audio[:max_length]
        elif len(audio) < max_length:
            return np.pad(audio, (0, max_length - len(audio)), 'constant')
        return audio

    @staticmethod
    def add_noise(audio, noise_factor=0.005):
        try:
            noise = np.random.randn(len(audio))
            augmented = audio + noise_factor * noise
            augmented = np.clip(augmented, -1.0, 1.0)
            return augmented
        except Exception as e:
            print(f"Warning: Error in add_noise: {str(e)}")
            return audio

    @staticmethod
    def time_shift(audio, shift_max=0.1):
        try:
            shift = int(len(audio) * shift_max)
            return np.roll(audio, shift) if shift > 0 else audio
        except Exception as e:
            print(f"Warning: Error in time_shift: {str(e)}")
            return audio

    @staticmethod
    def change_speed(audio, speed_factor=0.2):
        try:
            if not np.all(np.isfinite(audio)):
                audio = np.nan_to_num(audio, nan=0.0, posinf=1.0, neginf=-1.0)
            
            audio = np.clip(audio, -1.0, 1.0)
            speed_change = np.random.uniform(low=0.9, high=1.1)
            
            augmented = librosa.effects.time_stretch(audio, rate=speed_change)
            augmented = np.nan_to_num(augmented, nan=0.0, posinf=1.0, neginf=-1.0)
            augmented = np.clip(augmented, -1.0, 1.0)
            
            # Ensure fixed length after speed change
            augmented = AudioAugmenter.pad_or_truncate(augmented)
            
            return augmented
        except Exception as e:
            print(f"Warning: Error in change_speed: {str(e)}")
            return audio

    @staticmethod
    def augment(audio):
        # Ensure input is fixed length
        audio = AudioAugmenter.pad_or_truncate(audio)
        
        if not np.all(np.isfinite(audio)):
            audio = np.nan_to_num(audio, nan=0.0, posinf=1.0, neginf=-1.0)
            audio = np.clip(audio, -1.0, 1.0)
        
        augmentation_list = ['noise', 'shift', 'speed']
        num_augments = np.random.randint(1, 3)
        selected_augments = np.random.choice(augmentation_list, num_augments, replace=False)
        
        augmented = audio.copy()
        for aug_type in selected_augments:
            try:
                if aug_type == 'noise':
                    augmented = AudioAugmenter.add_noise(augmented)
                elif aug_type == 'shift':
                    augmented = AudioAugmenter.time_shift(augmented)
                elif aug_type == 'speed':
                    augmented = AudioAugmenter.change_speed(augmented)
                
                if not np.all(np.isfinite(augmented)):
                    augmented = np.nan_to_num(augmented, nan=0.0, posinf=1.0, neginf=-1.0)
                augmented = np.clip(augmented, -1.0, 1.0)
                
            except Exception as e:
                print(f"Warning: Error during {aug_type} augmentation: {str(e)}")
                continue
        
        # Final length check
        augmented = AudioAugmenter.pad_or_truncate(augmented)
        return augmented.astype(np.float32)

class ConsoleVisualizer:
    """Handles console-based visualization of plots"""
    @staticmethod
    def plot_confusion_matrix(cm, labels):
        print("\nConfusion Matrix:")
        print("-" * 40)
        
        # Header
        print(f"{'':>10}", end='')
        for label in labels:
            print(f"{label:>10}", end='')
        print("\n")
        
        # Matrix
        for i, label in enumerate(labels):
            print(f"{label:>10}", end='')
            for j in range(len(labels)):
                if cm[i][j] == 0:
                    color = Fore.WHITE
                elif cm[i][j] == np.max(cm[i]):
                    color = Fore.GREEN
                else:
                    color = Fore.YELLOW
                print(f"{color}{cm[i][j]:>10}{Style.RESET_ALL}", end='')
            print()
        print("-" * 40)

    @staticmethod
    def plot_training_history(history):
        print("\nTraining History:")
        print("-" * 40)

        # Extract relevant values from history
        train_losses = [entry["loss"] for entry in history if "loss" in entry]
        eval_losses = [entry["eval_loss"] for entry in history if "eval_loss" in entry]
        eval_accuracies = [entry["eval_accuracy"] for entry in history if "eval_accuracy" in entry]

        # Determine the number of epochs
        epochs = len(train_losses)  # Assuming loss is logged every epoch

        for epoch in range(epochs):
            print(f"Epoch {epoch+1:>2}: "
                  f"Loss: {train_losses[epoch]:.4f} "
                f"Val Loss: {eval_losses[epoch] if epoch < len(eval_losses) else 'N/A'} "
                f"Acc: {eval_accuracies[epoch] if epoch < len(eval_accuracies) else 'N/A'}")

class AudioDataset(TorchDataset):
    def __init__(self, audio_data, labels, feature_extractor, max_length=32000):
        self.audio_data = audio_data
        self.labels = labels
        self.feature_extractor = feature_extractor
        self.max_length = max_length
    
    def __len__(self):
        return len(self.labels)
    
    def pad_or_truncate(self, audio):
        """Pad or truncate audio to fixed length"""
        if len(audio) > self.max_length:
            return audio[:self.max_length]
        elif len(audio) < self.max_length:
            return np.pad(audio, (0, self.max_length - len(audio)), 'constant')
        return audio
    
    def __getitem__(self, idx):
        audio = self.audio_data[idx].astype(np.float32)
        # Ensure fixed length
        audio = self.pad_or_truncate(audio)
        
        inputs = self.feature_extractor(
            audio,
            sampling_rate=16000,
            padding=True,
            return_tensors="pt"
        )
        
        return {
            'input_values': inputs.input_values.squeeze(0),
            'label': torch.tensor(self.labels[idx])
        }

class AudioClassifier:
    def __init__(self, model_name="facebook/wav2vec2-base", num_labels=3):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = Wav2Vec2ForSequenceClassification.from_pretrained(
            model_name,
            num_labels=num_labels
        ).to(self.device)
        self.model.gradient_checkpointing_enable()
        self.feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(model_name)
        self.label_map = {'crying': 0, 'screaming': 1, 'normal': 2}
        self.config = self.model.config
        self.augmenter = AudioAugmenter()
        self.visualizer = ConsoleVisualizer()
        self.max_length = 32000  # 2 seconds at 16kHz

    @staticmethod
    def pad_or_truncate(audio, max_length=32000):
        """Pad or truncate audio to fixed length"""
        if len(audio) > max_length:
            return audio[:max_length]
        elif len(audio) < max_length:
            return np.pad(audio, (0, max_length - len(audio)), 'constant')
        return audio

    def load_audio_file(self, file_path, target_sr=16000):
        """Load and preprocess a single audio file"""
        try:
            audio, sr = librosa.load(
                file_path, 
                sr=target_sr, 
                mono=True,
            )
            # Normalize audio
            audio = audio / (np.max(np.abs(audio)) + 1e-6)
            # Ensure fixed length
            audio = self.pad_or_truncate(audio, self.max_length)
            return audio.astype(np.float32)
        except Exception as e:
            print(f"Error processing {file_path}: {str(e)}")
            return None

    def prepare_dataset(self, data_dir, metadata_file, augment=False):
        """Prepare dataset with fixed length audio"""
        print(f"Loading metadata from {metadata_file}")
        if not os.path.exists(metadata_file):
            raise FileNotFoundError(f"Metadata file not found: {metadata_file}")
        
        df = pd.read_csv(metadata_file)
        print(f"Loaded {len(df)} entries from metadata")
        
        audio_data = []
        labels = []
        
        for _, row in tqdm(df.iterrows(), total=len(df), desc="Loading audio files"):
            file_path = os.path.join(data_dir, row['file_name'])
            if not os.path.exists(file_path):
                print(f"Warning: File not found: {file_path}")
                continue
                
            audio = self.load_audio_file(file_path)
            if audio is not None:
                audio_data.append(audio)
                labels.append(self.label_map[row['label']])
                
                if augment:
                    augmented_audio = self.augmenter.augment(audio)
                    # Ensure fixed length for augmented audio
                    augmented_audio = self.pad_or_truncate(augmented_audio, self.max_length)
                    audio_data.append(augmented_audio.astype(np.float32))
                    labels.append(self.label_map[row['label']])
        
        dataset = AudioDataset(audio_data, labels, self.feature_extractor, self.max_length)
        print(f"Created dataset with {len(dataset)} examples")
        return dataset

class AudioClassifier:
    def __init__(self, model_name="facebook/wav2vec2-base", num_labels=3):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = Wav2Vec2ForSequenceClassification.from_pretrained(
            model_name,
            num_labels=num_labels
        ).to(self.device)
        self.model.gradient_checkpointing_enable()
        self.feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(model_name)
        self.label_map = {'crying': 0, 'screaming': 1, 'normal': 2}
        self.config = self.model.config
        self.augmenter = AudioAugmenter()
        self.visualizer = ConsoleVisualizer()

    @staticmethod
    def load_audio_file(file_path, target_sr=16000, max_duration=10):
        """Load and preprocess a single audio file with duration limit"""
        try:
            audio, sr = librosa.load(
                file_path, 
                sr=target_sr, 
                mono=True, 
                duration=max_duration
            )
            # Normalize audio
            audio = audio / np.max(np.abs(audio))
            return audio.astype(np.float32)
        except Exception as e:
            print(f"Error processing {file_path}: {str(e)}")
            return None

    def prepare_dataset(self, data_dir, metadata_file, augment=False):
        """Prepare dataset with custom AudioDataset class"""
        print(f"Loading metadata from {metadata_file}")
        if not os.path.exists(metadata_file):
            raise FileNotFoundError(f"Metadata file not found: {metadata_file}")
        
        df = pd.read_csv(metadata_file)
        print(f"Loaded {len(df)} entries from metadata")
        
        audio_data = []
        labels = []
        
        for _, row in tqdm(df.iterrows(), total=len(df), desc="Loading audio files"):
            file_path = os.path.join(data_dir, row['file_name'])
            if not os.path.exists(file_path):
                print(f"Warning: File not found: {file_path}")
                continue
                
            audio = self.load_audio_file(file_path)
            if audio is not None:
                audio_data.append(audio)
                labels.append(self.label_map[row['label']])
                
                if augment:
                    augmented_audio = self.augmenter.augment(audio)
                    audio_data.append(augmented_audio.astype(np.float32))
                    labels.append(self.label_map[row['label']])
        
        dataset = AudioDataset(audio_data, labels, self.feature_extractor)
        print(f"Created dataset with {len(dataset)} examples")
        return dataset
    
    def save_model(self, path):
        """Save model and feature extractor"""
        os.makedirs(path, exist_ok=True)
    
        # Save model properly
        self.model.save_pretrained(path)
    
        # Save feature extractor
        self.feature_extractor.save_pretrained(path)
    
        # Save label map
        with open(os.path.join(path, "label_map.json"), "w") as f:
            json.dump(self.label_map, f)
        print(f"Model saved at {path}")

    def train_kfold(self, dataset, output_dir, n_splits=5, use_wandb=False):
        """Train using k-fold cross validation"""
        kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
        
        # Get all indices
        indices = np.arange(len(dataset))
        
        fold_metrics = []
        
        for fold, (train_idx, val_idx) in enumerate(kf.split(indices)):
            print(f"\nTraining Fold {fold + 1}/{n_splits}")
            
            # Create fold datasets
            train_fold = torch.utils.data.Subset(dataset, train_idx)
            val_fold = torch.utils.data.Subset(dataset, val_idx)
            
            # Train on this fold
            fold_output_dir = os.path.join(output_dir, f"fold_{fold + 1}")
            trainer = self.train(train_fold, val_fold, fold_output_dir, use_wandb)
            
            # Evaluate fold
            with torch.no_grad():  # Use no_grad() to save memory
                metrics = trainer.evaluate()
            fold_metrics.append(metrics)
            
            # Visualize fold results
            self.visualizer.plot_training_history(trainer.state.log_history)

            # Clear CUDA memory after training this fold
            del trainer  # Delete trainer to free up memory
            torch.cuda.empty_cache()
            gc.collect()
        
        # Print average metrics across folds
        print("\nAverage Metrics Across Folds:")
        avg_metrics = {
            key: np.mean([fold[key] for fold in fold_metrics])
            for key in fold_metrics[0].keys()
        }
        print(json.dumps(avg_metrics, indent=2))
        
        return avg_metrics
    
    def compute_metrics(self, pred):
        """Compute metrics for evaluation"""
        labels = pred.label_ids
        preds = pred.predictions.argmax(-1)
        
        precision, recall, f1, _ = precision_recall_fscore_support(
            labels, preds, average='weighted'
        )
        acc = accuracy_score(labels, preds)
        
        return {
            "accuracy": acc,
            "f1": f1,
            "precision": precision,
            "recall": recall
        }

    def train(self, train_dataset, val_dataset, output_dir, use_wandb=False):
        """Train with early stopping"""
        training_args = TrainingArguments(
            output_dir=output_dir,
            num_train_epochs=25,  # Increased epochs for early stopping
            per_device_train_batch_size=8,
            per_device_eval_batch_size=8,
            gradient_accumulation_steps=4,
            fp16 = True,
            evaluation_strategy="steps",
            eval_steps=100,
            logging_steps=50,
            learning_rate=3e-5,
            weight_decay=0.01,
            warmup_steps=500,
            save_steps=100,
            load_best_model_at_end=True,
            metric_for_best_model="accuracy",
            report_to="wandb" if use_wandb else "none",
        )

        # Add early stopping
        early_stopping = EarlyStoppingCallback(
            early_stopping_patience=3,
            early_stopping_threshold=0.01
        )

        trainer = Trainer(
            model=self.model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=val_dataset,
            compute_metrics=self.compute_metrics,
            callbacks=[early_stopping]
        )

        # Clear CUDA memory before training
        torch.cuda.empty_cache()

        trainer.train()
        return trainer

    def generate_performance_report(self, trainer, test_dataset, output_dir):
        """Generate and visualize performance metrics"""
        predictions = trainer.predict(test_dataset)
        preds = predictions.predictions.argmax(-1)
        labels = predictions.label_ids

        # Calculate metrics
        metrics = self.compute_metrics(predictions)
        
        # Generate and display confusion matrix
        cm = confusion_matrix(labels, preds)
        label_names = list(self.label_map.keys())
        self.visualizer.plot_confusion_matrix(cm, label_names)
        
        # Print metrics
        print("\nPerformance Metrics:")
        print("-" * 40)
        for metric, value in metrics.items():
            print(f"{metric}: {value:.4f}")
        
        # Save metrics
        with open(os.path.join(output_dir, 'metrics.json'), 'w') as f:
            json.dump(metrics, f, indent=4)

        return metrics

def main():
    output_dir = "model_output"
    os.makedirs(output_dir, exist_ok=True)
    
    try:
        print("Initializing classifier...")
        classifier = AudioClassifier()
        
        print("\nPreparing datasets with augmentation...")
        dataset = classifier.prepare_dataset(
            "Split_Data/train",
            "Split_Data/train_metadata.csv",
            augment=True  # Enable augmentation
        )
        
        # Train with k-fold cross validation
        print("\nStarting k-fold cross validation training...")
        metrics = classifier.train_kfold(dataset, output_dir, n_splits=5)
        
        # Save best model
        best_model_path = os.path.join(output_dir, "best_model")
        classifier.save_model(best_model_path)
        print(f"\nBest model saved at: {best_model_path}")
        
    except Exception as e:
        print(f"\nError during execution: {str(e)}")
        raise

if __name__ == "__main__":
    main()

2025-02-23 18:14:09.161382: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-23 18:14:09.285832: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740314649.334462    2925 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740314649.347980    2925 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-23 18:14:09.452147: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

Initializing classifier...


/home/pratyush/miniconda3/envs/tf_env/lib/python3.10/site-packages/transformers/configuration_utils.py:315: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Preparing datasets with augmentation...
Loading metadata from Split_Data/train_metadata.csv
Loaded 3129 entries from metadata


Loading audio files:   0%|          | 0/3129 [00:00<?, ?it/s]

...

Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
100,0.061200,0.070544,0.987210,0.987240,0.987366,0.987210
200,0.070800,0.044151,0.988809,0.988815,0.988835,0.988809
300,0.112000,0.052872,0.983213,0.983220,0.983408,0.983213
400,0.098200,0.113385,0.972022,0.971954,0.972815,0.972022



Training History:
----------------------------------------
Epoch  1: Loss: 0.0600 Val Loss: 0.0705437883734703 Acc: 0.9872102318145484
Epoch  2: Loss: 0.0612 Val Loss: 0.044151317328214645 Acc: 0.9888089528377298
Epoch  3: Loss: 0.0759 Val Loss: 0.0528721921145916 Acc: 0.9832134292565947
Epoch  4: Loss: 0.0708 Val Loss: 0.11338482052087784 Acc: 0.9720223820943246
Epoch  5: Loss: 0.0725 Val Loss: 0.044151317328214645 Acc: 0.9888089528377298
Epoch  6: Loss: 0.1120 Val Loss: N/A Acc: N/A
Epoch  7: Loss: 0.0839 Val Loss: N/A Acc: N/A
Epoch  8: Loss: 0.0982 Val Loss: N/A Acc: N/A

Training Fold 5/5


/home/pratyush/miniconda3/envs/tf_env/lib/python3.10/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
100,0.065100,nan,0.987210,0.987234,0.987540,0.987210
200,0.030200,nan,0.992806,0.992812,0.992842,0.992806
300,0.092100,nan,0.984013,0.984040,0.984400,0.984013
400,0.100400,nan,0.984812,0.984812,0.984813,0.984812



Training History:
----------------------------------------
Epoch  1: Loss: 0.0400 Val Loss: nan Acc: 0.9872102318145484
Epoch  2: Loss: 0.0651 Val Loss: nan Acc: 0.9928057553956835
Epoch  3: Loss: 0.0299 Val Loss: nan Acc: 0.9840127897681854
Epoch  4: Loss: 0.0302 Val Loss: nan Acc: 0.9848121502797762
Epoch  5: Loss: 0.0616 Val Loss: nan Acc: 0.9928057553956835
Epoch  6: Loss: 0.0921 Val Loss: N/A Acc: N/A
Epoch  7: Loss: 0.0922 Val Loss: N/A Acc: N/A
Epoch  8: Loss: 0.1004 Val Loss: N/A Acc: N/A

Average Metrics Across Folds:
{
  "eval_loss": NaN,
  "eval_accuracy": 0.9790705454805485,
  "eval_f1": 0.9790674440522349,
  "eval_precision": 0.9792200873731648,
  "eval_recall": 0.9790705454805485,
  "eval_runtime": 6.65872,
  "eval_samples_per_second": 189.3056,
  "eval_steps_per_second": 23.7458,
  "epoch": 3.313738019169329
}
Model saved at model_output/best_model

Best model saved at: model_output/best_model
